## 3차 과제 - 당뇨병을 예측해보자
- 주제: 데이터 분석 실습(머신러닝)
- 데이터: 피마 인디언 당뇨병 데이터
https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
- 사용 라이브러리: numpy, pandas, matplotlib, sklearn


### 1. EDA, 전처리

1.1 분석을 위해 사용될 라이브러리들을 import 하세요.

numpy, pandas, matplotlib, sklearn(train_test_split, accurarcy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve, StandardScaler, LogisticRegression)

In [68]:
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import StandardScaler


1.2 데이터를 불러오고 타겟 변수('Outcome')에 속한 값들의 비율과(value_counts 활용) 전체 데이터 앞 7개 값을 확인하세요.

In [69]:
df = pd.read_csv("diabetes.csv")

In [70]:
df.head(7)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1


1.3 데이터의 feature 타입과 Null 개수를 확인하세요.

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [72]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

1.4 예측을 위해 데이터를 feature 데이터 셋과 target 데이터 셋으로 나누세요.

In [73]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

1.5 X와 y로 나눈 데이터 셋을 다시 train, test로 나누세요(test_size = 0.3, random_state=123, stratify=y).

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

1.6 test size는 어느정도로 해야 좋은지 그리고 왜 그래야 하는지 작성하세요.

큰 데이터셋일수록 테스트 세트의 비율을 늘려도 상관없지만, 작은 데이터셋이라면 가능한 작은 테스트셋을 잡아놓는게 좋다. 일반적으로는 25%에서 30%정도가 적당하다고 본다.

1.7 startify 값을 설정하는 이유와 언제 주로 사용되는지 서술하세요.

startify는 데이터를 나눌 때 각 클래스의 비율을 유지하기 위해 사용되는 매개변수이고, 데이터를 나눌 때 각 클래스의 비율을 유지하면 모델의 성능을 더욱 정확하게 평가할 수 있다. 이는 특히 불균형한 클래스 분포를 갖는 데이터셋에서 중요하다.

### 2. 예측 및 분석

2.1 성능 평가에 사용될 함수를 만들어보겠습니다. accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score로 구성되어 있으며 이를 한번에 출력할 수 있도록 합니다.

In [75]:
def clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬,\n', confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:4f}, F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

2. 위에서 사용한 성능지표인 Confusion Matrix에 대해 설명하세요.

In [77]:
Confusion Matrix는 머신러닝 및 통계에서 분류 모델의 성능을 평가하는 데 사용되는 행렬이고, 
모델이 예측한 결과와 실제 레이블 간의 관계를 보여준다.

SyntaxError: invalid syntax (2908720682.py, line 1)

2.2 로지스틱 회귀로 예측 후 위에서 만든 clf_eval() 함수를 통해 잘 예측했는지 확인하세요(solver='linear'로 설정, 예측 변수명은 pred, pred_proba).

In [78]:
model = LogisticRegression(solver='linear')
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:,1]
# 버전 문제 때문에 주피터에서는 안돌아가는듯함
# 코랩에서도 자꾸 에러가 나서 일단은 이 상태로 제출하겠슴다..

InvalidParameterError: The 'solver' parameter of LogisticRegression must be a str among {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'saga', 'sag'}. Got 'linear' instead.

In [79]:
clf_eval(y_test, pred, pred_proba)

NameError: name 'pred' is not defined

2.3 LogisticRegression에서 solver 파라미터의 역할과 어떤 것들이 있고 각각은 무엇을 의미하는지 작성하세요.

solver 매개변수는 로지스틱 회귀 모델의 최적화에 사용되고, 우리가 쓴 liblinear 솔버는 작은 데이터셋에서의 이진 분류에 적합하고 newton-cg 또한 작은 데이터셋에 적합. 대규모 데이터셋에서는 lbfgs가 적합하다.

2.4 pred와 pred_proba는 각각 어떤 차이가 있는지 작성하세요.

pred 는 예측된 레이블에 적용하고, pred_proba 는 예측된 레이블이 없고 예측 확률로 나타내어져 있을 때 쓴다.

2.5 전체 데이터 중 약 65%가 Negative이므로 재현율 성능에 조첨을 맞추겠습니다. 정밀도 재현율 curve를 보고 임계값 별 정밀도 재현율 값의 변화를 확인하기 위해 이를 위한 함수를 작성하겠습니다.

In [80]:
def precision_recall_curve_plot(y_test, pred_proba_c1):
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    plt.xlabel('Threshold value')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.show()

2.6 작성한 함수를 바탕으로 임계값 별 정밀도와 재현율 값 변화를 확인하세요.

In [81]:
pred_proba_c1 = model.predict_proba(X_test)[:,1]
precision_recall_curve_plot(y_test, pred_proba_c1)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

2.7 그래프를 통해 확인했을 때 Threshold를 0.38정도로 설정했을 때 recall과 precision이 어느정도 균형이 맞음을 알 수 있습니다. 그러나 두 지표 모두 0.7 이하로 낮은 값이기에 다시 확인하도록 하겠습니다. 먼저 describe() 함수를 통해 개별 feature에 대한 분포도를 확인하세요.

In [82]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


2.8 위에서 확인해봤을 때 min 값이 0으로 되어 있는 feature가 다수 존재합니다. Glucose(포도당 수치), BMI 지수가 0인 것은 말이 안 되므로 이들을 평균값으로 대체하세요.

In [83]:
### Glucose, BloodPressure, SkinThickness, Insulin, BMI에 대해 평균값으로 대체
zero_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
mean_zero_features = df[zero_features].mean()
for features in zero_features:
    df[zero_features] = df[zero_features].replace(0, mean_zero_features)

2.9 이후 대체한 feature들에 대해 feature scaling을 진행하겠습니다. 모두 numeric feature이므로 X에 대해 StandardScaler를 일괄 적용하세요.

In [84]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

2.10 feature scaling은 왜 사용하고 어떤 것들이 있는지 작성하세요.

feature 들 간의 값들의 양 자체에 유의미한 차이가 있으면 그 크기에 따라서 반영이 크게 될 수도, 적게 될 수도 있기 때문에 적합한 모델을 구축하기 위해서 normalization 하는 용도로 쓴다.

2.11 다시 에측하기에 앞서 위에서 scaling을 통해 변경된 X_scaled를 활용해 다시 train_test_split을 진행하세요(함수 파라미터 동일).

In [85]:
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=123, stratify=y)


2.12 로지스틱 회귀로 앞서 진행한 것과 동일하게 진행하세요.

In [86]:
model = LogisticRegression(solver='linear')
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:,1]

InvalidParameterError: The 'solver' parameter of LogisticRegression must be a str among {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'saga', 'sag'}. Got 'linear' instead.

In [87]:
clf_eval(y_test, pred, pred_proba)

NameError: name 'pred' is not defined

### 3. 자유 분석
성능을 높일 수 있는 방법들을 활용해 예측을 진행하세요.

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import tqdm

dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:, -1:]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

Xtrain = X[:700]
ytrain = y[:700]
Xtest = X[700:]
ytest = y[700:]

model = nn.Sequential(
    nn.Linear(8,12), 
    nn.ReLU(), 
    nn.Linear(12,8), 
    nn.ReLU(), 
    nn.Linear(8,1), 
    nn.Sigmoid())

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 500
batch_size = 20
batches_for_epoch = len(Xtrain) // batch_size            # //는 소수점 밑 버러기

train_loss= []
train_acc =[]
test_acc =[]

for epoch in range(n_epochs):
    for i in range(batches_for_epoch):
    # for i in range(0, len(Xtrain), batch_size):
        start = i*batch_size
        Xbatch = Xtrain[start:start+batch_size]
        ybatch = ytrain[start:start+batch_size]
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        train_loss.append(loss)
        acc = (y_pred.round()==ybatch).float().mean()
        train_acc.append(acc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'epoch {epoch} step {i} loss {loss} acc {acc}')
    print(f'Finished epoch {epoch}, latest loss {loss}')
    with torch.no_grad():
        y_pred= model(Xtest)

    accuracy = (y_pred.round()==ytest).float().mean()
    test_acc.append(accuracy)
    print(f'Accuracy {accuracy}')

epoch 0 step 0 loss 2.286107301712036 acc 0.3499999940395355
epoch 0 step 1 loss 1.0765231847763062 acc 0.6000000238418579
epoch 0 step 2 loss 0.8149967193603516 acc 0.699999988079071
epoch 0 step 3 loss 1.0025396347045898 acc 0.550000011920929
epoch 0 step 4 loss 0.7508574724197388 acc 0.699999988079071
epoch 0 step 5 loss 0.9614806175231934 acc 0.6499999761581421
epoch 0 step 6 loss 0.9515274167060852 acc 0.5
epoch 0 step 7 loss 0.7717649340629578 acc 0.6000000238418579
epoch 0 step 8 loss 0.740638017654419 acc 0.550000011920929
epoch 0 step 9 loss 0.9823617935180664 acc 0.5
epoch 0 step 10 loss 0.8469031453132629 acc 0.3499999940395355
epoch 0 step 11 loss 0.872628390789032 acc 0.44999998807907104
epoch 0 step 12 loss 0.6956683397293091 acc 0.5
epoch 0 step 13 loss 0.8043657541275024 acc 0.5
epoch 0 step 14 loss 0.7641310691833496 acc 0.4000000059604645
epoch 0 step 15 loss 0.7819724082946777 acc 0.4000000059604645
epoch 0 step 16 loss 0.7314140200614929 acc 0.44999998807907104
epoc

In [ ]:
#이런 간단한 Sequential 모델이어도 에포크 500정도만 돌리면 75%정도 정확도가 나오는게 신기하면서도  test,train 데이터셋 나누고 normalization 제대로 한 다음 돌리면 훨씬 좋아질거란 생각이 들었다!